In [ ]:
'''
Script calculating the a P3 value from single trials for each channel (or for GFP)
Generates a plot comparing the changes in amplitudes vs. in conditions over time
'''

In [1]:
import numpy as np
import os
import scipy.io as io
import matplotlib.pyplot as plt
from ms2time import ms2time


def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def find_outliers(data):
    data_mean, data_std = np.mean(data), np.std(data)
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off
    outliers = [x for x in range(len(data)) if data[x] < lower or data[x] > upper]
    return outliers


In [2]:
directory_ = '/home/data/Project_ASD/clean-trials/'
directory_ = '/home/data/Project_ASD/3Dmat4ML/'
groups = ['Autism', 'Neurotypical']
baseMethods=['','baseline50','baseline100']
baseMethod=baseMethods[1]
targetcodes=[13,16,23,26,33,36,43,46]


In [3]:
subjects=np.empty((0,3),int) # group ind, subject ind, subject ID - e.g.['0' '5' '1888']

s=0
for g in range((len(groups))):
    list_gsubj = list(os.listdir(directory_+'/'+groups[g]))
    for subject in list_gsubj:
        if subject[0]=='1':
            subjects = np.append(subjects,[[g,s,subject]], axis=0)
            s+=1
n_subj=subjects.shape[0]

In [5]:

targetP3s = np.zeros((n_subj,2), dtype=object)
for g,s,subject in subjects:

    subjectData = io.loadmat(directory_+groups[int(g)]+'/'+subject)
    conds = subjectData['evConds']
    events = subjectData['events']
    ERPs = subjectData['ERPs'].astype('float16') 
    preStim= subjectData['preStim'].astype('float16')
    baseline= np.mean(preStim,1)
    ERPs_nobase=np.zeros((122,160,ERPs.shape[2]),dtype=object)
    for t in range(122):
        ERPs_nobase[t]=np.take(ERPs, t, axis=1)-baseline
                
    ind_target = [] 
    for i in range(0, len(events[0])) : 
        if events[0,i] in targetcodes : 
            ind_target.append(i) 
    ERPs=np.array(ERPs, dtype=np.float64)
    targERPs=np.take(ERPs,ind_target,axis=2)
    targP3s=np.take(targERPs,ms2time(250,400),axis=1)
    targP3s=np.mean(targP3s,axis=1)
    targetP3s[int(s),0]=targP3s

    targConds=np.take(conds,ind_target,axis=1)
    targConds=np.squeeze(targConds)
    targetP3s[int(s),1]=targConds

    print(subject)



11354.mat
1134.mat
1190.mat
11913.mat


/home/seyda/.local/lib/python3.6/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/home/seyda/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in subtract
  if sys.path[0] == '':
/home/seyda/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in subtract
  if sys.path[0] == '':


1160.mat
1179.mat
11160.mat
1838.mat
1132.mat
11723.mat
11076.mat
1888.mat
11673.mat
1106.mat
11051.mat
11275.mat
1108.mat
11750.mat
1154.mat
1174.mat
11977.mat
12778.mat
12727.mat
12694.mat
12413.mat
10369.mat
10438.mat
12516.mat
12005.mat
10257.mat
12360.mat
12497.mat
10056.mat
12563.mat
10594.mat
10545.mat
12433.mat
12328.mat
12562.mat
12443.mat
12512.mat
10585.mat
12386.mat
10049.mat
12414.mat


In [6]:
#Save your results 
directory = '/home/data/Project_ASD/' + os.environ["USER"]+'/' #feel free to upload data from /Project_ASD/seyda
#np.savez(directory+'targetP3s.npz', targetP3s=targetP3s, subjects=subjects)
